In [5]:
import re
from collections import defaultdict
import itertools

def findsubsets(s, n):
    return list(itertools.combinations(s, n))

def read_data():
    file1 = open('data-2.txt', 'r')
    Lines = file1.readlines()
    t = []
    for line in Lines:
        listM = tuple(int (i) for i in re.findall('[0-9]+', line))
        t.append(listM)
    
    return t
    
def read_params():
    file2 = open('para-2.txt', 'r')
    M_Lines = file2.readlines()
    MIS = dict()
    rest = -1
    SDC = -1
    for line in M_Lines:
        if line!='\n':
            lhs, rhs = line.split("=", 1)
            ls = re.search(r"\((.*?)\)|SDC", lhs)
            if(lhs.strip()=="SDC"): 
                SDC = float(rhs)
            elif(ls.group(1)=="rest"): rest = float(rhs)
            else: MIS[int(ls.group(1))] = float(rhs)
            
    return MIS, rest, SDC
        
def sort(MIS, rest, t):
    counts = {}
    counts = defaultdict(lambda:0,counts)
    for t in transactions:
        for val in t:
            counts[val]+=1
            if(val not in MIS): MIS[val] = rest

    MIS = dict(sorted(MIS.items(), key = lambda kv: kv[1]))
    n = len(transactions)
    
    return counts, MIS, n

def init_pass(MIS,counts):
    L = []
    flag = 0
    for k in MIS:
        if(flag==0 and counts[k]/n >= MIS[k]):
            flag = 1
            L.append(k)
            i=k
        elif(flag==1 and counts[k]/n >= MIS[i]):
            L.append(k)
            
    return L

def level2_candidate_gen(L,SDC,MIS,counts,n):
    C2 = set()
    for l, val in enumerate(L):
        if(counts[val]/n >= MIS[val]): 
            r = l;
            sup_r = counts[val]/n
            
            for j in range(r+1, len(L)):
                if(counts[L[j]]/n >= MIS[val]):
                    sup_j = counts[L[j]]/n
                    if(SDC != -1):
                        if(abs(sup_r - sup_j) <= SDC):
                            if(MIS[val]==MIS[L[j]]):
                                C2.add(tuple((min(val,L[j]),max(val,L[j]))))
                            else:
                                C2.add(tuple((val,L[j])))
                    else:
                        if(MIS[val]==MIS[L[j]]):
                            C2.add(tuple((min(val,L[j]),max(val,L[j]))))
                        else:
                            C2.add(tuple((val,L[j])))
    return C2

def levelk_candidate_gen(F_temp,SDC,MIS,counts,n,k):
    Ck = set()
    J = findsubsets(F_temp,2)
    #print(J)
    for j in J:
        if(j[0][:-1] == j[1][:-1]):
            #print(j, j[0][:-1] , j[1][:-1])
            a = j[0][-1]
            b = j[1][-1]
            #print(a,b)
            sup_a = counts[a]/n
            sup_b = counts[b]/n
            #print(sup_a, sup_b)
            if(SDC!=-1):
                if(abs(sup_a - sup_b) <= SDC):
                    if(MIS[a]==MIS[b]):
                        Ck.add(tuple(((j[0][:-1]+(min(a,b),)+(max(a,b),)))))
                    elif(MIS[a]>MIS[b]):
                        Ck.add(tuple(((j[0][:-1]+(b,)+(a,)))))
                    else:
                        Ck.add(tuple(((j[0][:-1]+(a,)+(b,)))))
            else:
                if(MIS[a]==MIS[b]):
                    Ck.add(tuple(((j[0][:-1]+(min(a,b),)+(max(a,b),)))))
                elif(MIS[a]>MIS[b]):
                    Ck.add(tuple(((j[0][:-1]+(b,)+(a,)))))
                else:
                    Ck.add(tuple(((j[0][:-1]+(a,)+(b,)))))
    
    if(Ck):
        for c in Ck.copy():
            S = findsubsets(c,k-1)

            for s in S:
                if(c[0] in s or MIS[c[0]] == MIS[c[1]]):
                    if(s not in F_temp):
                        Ck.remove(c)
                        break
    return Ck

def convertTuple(tup):
    s = '       ('
    for item in tup:
        s = s + str(item) + ' '
    s = s[:-1] + ')\n'
    return s


def write_output(F):
    s = '76 60\n'
    for i in F:
        s = s + "(Length-"+ str(i) + " " + str(len(F[i]))  + "\n"
        for j in F[i]:
            s = s + convertTuple(j)
        s = s + ')\n' 
    
    with open("result2.txt", "w") as text_file:
        text_file.write(s)
    
    #print(s)


if __name__ == '__main__':
    transactions = read_data()
    MIS, rest, SDC = read_params()
    counts, MIS, n = sort(MIS, rest,transactions)
    L = init_pass(MIS,counts)
    
    F = dict()
    F1 = []
    for k in L:
        if(counts[k]/n >= MIS[k]):
            F1.append(tuple((k,)))
    F[1] = F1
    
    k = 2
    while(k-1 in F):
        Fk = []
        if k==2:
            Ck = level2_candidate_gen(L,SDC,MIS,counts,n)

        else:
            Ck = levelk_candidate_gen(set(F[k-1]),SDC,MIS,counts,n,k)

        if(Ck):
            for t in transactions:
                for c in Ck:
                    if(set(c).issubset(t)):
                        counts[c]+=1
                    if(set(c[1:]).issubset(t)):
                        counts[c[1:]]+=1

            for c in Ck:
                if(counts[c]/n >= MIS[c[0]]):
                    Fk.append(c)


        if(Fk):
            F[k] = Fk
        k+=1
        
    write_output(F)